In [1]:
import numpy as np
print("Loading training data ...")
data_train = np.genfromtxt("data/LSDA2017GalaxiesTrain.csv",delimiter=",", dtype=float)
Xtrain, ytrain = data_train[:,:-1], data_train[:,-1]
print("Loaded training data: n=%i, d=%i" % (Xtrain.shape[0], Xtrain.shape[1]))

data_validation = np.genfromtxt("data/LSDA2017GalaxiesValidate.csv",delimiter=",", dtype=float)
Xvd, yvd = data_validation[:,:-1], data_validation[:,-1]
print("Loaded validation data: n=%i, d=%i" % (Xvd.shape[0], Xvd.shape[1]))

data_test = np.genfromtxt("data/LSDA2017GalaxiesTest.csv",delimiter=",", dtype=float)
Xtest, ytest = data_test[:,:-1], data_test[:,-1]
print("Loaded test data: n=%i, d=%i" % (Xtest.shape[0], Xtest.shape[1]))

Loading training data ...
Loaded training data: n=4000, d=18
Loaded validation data: n=1000, d=18
Loaded test data: n=5000, d=18


In [2]:
ytrain_var = np.std(ytrain)**2
print("Standard deviation for train label is %f" % ytrain_var)

def tf_input(X,y):
    # Preprocess your data here...

    #X = tf.constant(X)
    #y = tf.constant(y)
    return X, y

Standard deviation for train label is 0.010498


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import shutil
import os

# Splits data into mini-batches
# Batches are not randomized/shuffled, shuffling the data in mini-batch learning typically improves the performance
class Batcher:
    'Splits data into mini-batches'
    def __init__(self, data, batchSize):
        self.data, self.target = data
        self.batchSize = batchSize
        self.batchStartIndex = 0
        self.batchStopIndex = 0
        self.noData = self.data.shape[0]
    def nextBatch(self):
        self.batchStartIndex = self.batchStopIndex % self.noData
        self.batchStopIndex = min(self.batchStartIndex + self.batchSize, self.noData)
        return self.data[self.batchStartIndex:self.batchStopIndex], self.target[self.batchStartIndex:self.batchStopIndex]

# Flags
try:
   FLAGS
except NameError:
    flags = tf.app.flags
    FLAGS = flags.FLAGS
    flags.DEFINE_string('summary_dir', 'tensor_logs', 'directory to put the summary data')
    flags.DEFINE_string('data_dir', 'data', 'directory with data')
    flags.DEFINE_integer('maxIter', 100000, 'number of iterations')
    flags.DEFINE_integer('batchSize', 128, 'batch size')
    flags.DEFINE_integer('noHidden1', 64, 'size of first hidden layer')
    flags.DEFINE_integer('noHidden2', 32, 'size of second hidden layer')
    flags.DEFINE_float('lr', 0.001, 'initial learning rate')
    # when we reload the flags, we clear the output dir
    if not os.path.exists(flags.FLAGS.summary_dir):
        os.makedirs(flags.FLAGS.summary_dir)
    else:
        shutil.rmtree(flags.FLAGS.summary_dir)
        os.makedirs(flags.FLAGS.summary_dir)


    
X_train ,y_train = tf_input(Xtrain,ytrain)
# Number of training data points
noTrain = X_train.shape[0]
print("Numer of training data points:", noTrain)

# Input dimension
inDim = X_train.shape[1]

X_vd,y_vd = tf_input(Xvd,yvd)
X_test ,y_test = tf_input(Xtest,ytest)

Numer of training data points: 4000


In [4]:
shutil.rmtree(flags.FLAGS.summary_dir)

# Create graph
sess = tf.Session()

# Initialize placeholders
x_data = tf.placeholder(shape=[None, inDim], dtype=tf.float32, name='input')
y_target = tf.placeholder(shape=[None, 1], dtype=tf.float32, name='target')

# Define variables
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=np.std(ytrain)) # restrict to +/- 2*stddev
    return tf.Variable(initial, name='weights')

def bias_variable(shape):
    initial = tf.truncated_normal(shape, stddev=np.std(ytrain))
    return tf.Variable(initial, name='bias')


# Define model
with tf.name_scope('layer1') as scope:
    W_1 = weight_variable([inDim, FLAGS.noHidden1])
    b_1 = bias_variable([FLAGS.noHidden1])
    y_1 = tf.matmul(x_data , W_1 ) + b_1
    y_1 = tf.nn.relu(y_1)
    

with tf.name_scope('layer2') as scope:
    W_2 = weight_variable([FLAGS.noHidden1, 1])
    b_2 = bias_variable([1])
    model_output = tf.matmul(y_1, W_2) + b_2


# Declare loss function
loss = tf.reduce_mean(tf.square(model_output - y_target), name='mean_squared_error')
tf.summary.scalar('loss', loss)

# Declare optimizer
my_opt =  tf.train.GradientDescentOptimizer(FLAGS.lr)
train_step = my_opt.minimize(loss)


# Logging
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(FLAGS.summary_dir + '/train')
test_writer = tf.summary.FileWriter(FLAGS.summary_dir + '/test')
validate_writer=  tf.summary.FileWriter(FLAGS.summary_dir + '/validate')
writer = tf.summary.FileWriter(FLAGS.summary_dir, sess.graph)
saver = tf.train.Saver() # for storing the best network



In [5]:
# Initialize variables
init = tf.global_variables_initializer()
sess.run(init)

# Best validation accuracy seen so far
bestValidation = np.inf

# Mini-batches for training
batcher = Batcher((X_train,y_train), FLAGS.batchSize)

# Training loop
for i in range(FLAGS.maxIter):
    xTrain, yTrain = batcher.nextBatch()
    sess.run(train_step, feed_dict={x_data: xTrain, y_target: np.transpose([yTrain])})
    summary = sess.run(merged, feed_dict={x_data: xTrain, y_target: np.transpose([yTrain])})
    train_writer.add_summary(summary, i)
    if((i+1)%100==0):
        print("Iteration:",i+1,"/",FLAGS.maxIter)
        #summary = sess.run(merged, feed_dict={x_data: X_test, y_target: np.transpose([y_test])})
        #test_writer.add_summary(summary, i)
        currentValidation, summary = sess.run([loss, merged], feed_dict={x_data: X_vd, y_target: np.transpose([y_vd])})
        validate_writer.add_summary(summary, i)
        if(currentValidation < bestValidation):
            bestValidation = currentValidation
            saver.save(sess=sess, save_path=FLAGS.summary_dir + '/bestNetwork')
            print("\tbetter network stored,",currentValidation,"<",bestValidation)

# Print values after last training step
print("final training accuracy:", sess.run(loss, feed_dict={x_data: X_train, y_target: np.transpose([y_train])}),
      "final test accuracy: ", sess.run(loss, feed_dict={x_data: X_test, y_target: np.transpose([y_test])}),
      "final validation accuracy: ", sess.run(loss, feed_dict={x_data: X_vd, y_target: np.transpose([y_vd])}))

# Load the network with the lowest validation error
saver.restore(sess=sess, save_path=FLAGS.summary_dir + '/bestNetwork')
print("best training accuracy:", sess.run(loss, feed_dict={x_data: X_train, y_target: np.transpose([y_train])}),
      "best test accuracy: ", sess.run(loss, feed_dict={x_data: X_test, y_target: np.transpose([y_test])}),
      "best validation accuracy: ", sess.run(loss, feed_dict={x_data: X_vd, y_target: np.transpose([y_vd])}))


Iteration: 100 / 100000
	better network stored, 0.0196421 < 0.0196421
Iteration: 200 / 100000
	better network stored, 0.0167084 < 0.0167084
Iteration: 300 / 100000
	better network stored, 0.0144268 < 0.0144268
Iteration: 400 / 100000
	better network stored, 0.0128181 < 0.0128181
Iteration: 500 / 100000
	better network stored, 0.0117126 < 0.0117126
Iteration: 600 / 100000
	better network stored, 0.0109608 < 0.0109608
Iteration: 700 / 100000
	better network stored, 0.0101286 < 0.0101286
Iteration: 800 / 100000
	better network stored, 0.009946 < 0.009946
Iteration: 900 / 100000
	better network stored, 0.00901575 < 0.00901575
Iteration: 1000 / 100000
	better network stored, 0.00860397 < 0.00860397
Iteration: 1100 / 100000
	better network stored, 0.00830101 < 0.00830101
Iteration: 1200 / 100000
	better network stored, 0.00795361 < 0.00795361
Iteration: 1300 / 100000
	better network stored, 0.00771528 < 0.00771528
Iteration: 1400 / 100000
	better network stored, 0.00754683 < 0.00754683
Itera

Iteration: 13800 / 100000
	better network stored, 0.00216393 < 0.00216393
Iteration: 13900 / 100000
Iteration: 14000 / 100000
	better network stored, 0.00215628 < 0.00215628
Iteration: 14100 / 100000
	better network stored, 0.00214642 < 0.00214642
Iteration: 14200 / 100000
Iteration: 14300 / 100000
	better network stored, 0.00214533 < 0.00214533
Iteration: 14400 / 100000
Iteration: 14500 / 100000
	better network stored, 0.00212303 < 0.00212303
Iteration: 14600 / 100000
	better network stored, 0.00211744 < 0.00211744
Iteration: 14700 / 100000
Iteration: 14800 / 100000
	better network stored, 0.00211224 < 0.00211224
Iteration: 14900 / 100000
	better network stored, 0.00210285 < 0.00210285
Iteration: 15000 / 100000
Iteration: 15100 / 100000
	better network stored, 0.00210212 < 0.00210212
Iteration: 15200 / 100000
Iteration: 15300 / 100000
	better network stored, 0.0020833 < 0.0020833
Iteration: 15400 / 100000
	better network stored, 0.00207773 < 0.00207773
Iteration: 15500 / 100000
Iterat

Iteration: 32400 / 100000
Iteration: 32500 / 100000
Iteration: 32600 / 100000
Iteration: 32700 / 100000
	better network stored, 0.00176237 < 0.00176237
Iteration: 32800 / 100000
Iteration: 32900 / 100000
Iteration: 33000 / 100000
	better network stored, 0.00176115 < 0.00176115
Iteration: 33100 / 100000
Iteration: 33200 / 100000
Iteration: 33300 / 100000
Iteration: 33400 / 100000
Iteration: 33500 / 100000
	better network stored, 0.00175531 < 0.00175531
Iteration: 33600 / 100000
Iteration: 33700 / 100000
Iteration: 33800 / 100000
	better network stored, 0.00175468 < 0.00175468
Iteration: 33900 / 100000
Iteration: 34000 / 100000
Iteration: 34100 / 100000
Iteration: 34200 / 100000
Iteration: 34300 / 100000
	better network stored, 0.0017488 < 0.0017488
Iteration: 34400 / 100000
Iteration: 34500 / 100000
Iteration: 34600 / 100000
	better network stored, 0.00174816 < 0.00174816
Iteration: 34700 / 100000
Iteration: 34800 / 100000
Iteration: 34900 / 100000
Iteration: 35000 / 100000
Iteration: 3

Iteration: 57600 / 100000
Iteration: 57700 / 100000
Iteration: 57800 / 100000
Iteration: 57900 / 100000
Iteration: 58000 / 100000
Iteration: 58100 / 100000
Iteration: 58200 / 100000
Iteration: 58300 / 100000
	better network stored, 0.00158377 < 0.00158377
Iteration: 58400 / 100000
Iteration: 58500 / 100000
Iteration: 58600 / 100000
Iteration: 58700 / 100000
Iteration: 58800 / 100000
Iteration: 58900 / 100000
Iteration: 59000 / 100000
Iteration: 59100 / 100000
	better network stored, 0.00157912 < 0.00157912
Iteration: 59200 / 100000
Iteration: 59300 / 100000
Iteration: 59400 / 100000
Iteration: 59500 / 100000
Iteration: 59600 / 100000
Iteration: 59700 / 100000
Iteration: 59800 / 100000
Iteration: 59900 / 100000
	better network stored, 0.00157487 < 0.00157487
Iteration: 60000 / 100000
Iteration: 60100 / 100000
Iteration: 60200 / 100000
Iteration: 60300 / 100000
Iteration: 60400 / 100000
Iteration: 60500 / 100000
Iteration: 60600 / 100000
Iteration: 60700 / 100000
	better network stored, 

Iteration: 83500 / 100000
Iteration: 83600 / 100000
Iteration: 83700 / 100000
Iteration: 83800 / 100000
Iteration: 83900 / 100000
	better network stored, 0.00145823 < 0.00145823
Iteration: 84000 / 100000
Iteration: 84100 / 100000
Iteration: 84200 / 100000
Iteration: 84300 / 100000
Iteration: 84400 / 100000
Iteration: 84500 / 100000
Iteration: 84600 / 100000
Iteration: 84700 / 100000
	better network stored, 0.00145511 < 0.00145511
Iteration: 84800 / 100000
Iteration: 84900 / 100000
Iteration: 85000 / 100000
Iteration: 85100 / 100000
Iteration: 85200 / 100000
Iteration: 85300 / 100000
Iteration: 85400 / 100000
Iteration: 85500 / 100000
	better network stored, 0.00145158 < 0.00145158
Iteration: 85600 / 100000
Iteration: 85700 / 100000
Iteration: 85800 / 100000
Iteration: 85900 / 100000
Iteration: 86000 / 100000
Iteration: 86100 / 100000
Iteration: 86200 / 100000
Iteration: 86300 / 100000
	better network stored, 0.00144843 < 0.00144843
Iteration: 86400 / 100000
Iteration: 86500 / 100000
It